## **eBay car sales** guided project
---
> Foreword:
> In this project I will implement new (for me) data cleaning technics I've learned lately. Just enjoy my code and solutions)

The data dictionary provided with data is as follows:

* dateCrawled - When this ad was first crawled. All field-values are taken from this date.
* name - Name of the car.
* seller - Whether the seller is private or a dealer.
* offerType - The type of listing
* price - The price on the ad to sell the car.
* abtest - Whether the listing is included in an A/B test.
* vehicleType - The vehicle Type.
* yearOfRegistration - The year in which the car was first registered.
* gearbox - The transmission type.
* powerPS - The power of the car in PS.
* model - The car model name.
* kilometer - How many kilometers the car has driven.
* monthOfRegistration - The month in which the car was first registered.
* fuelType - What type of fuel the car uses.
* brand - The brand of the car.
* notRepairedDamage - If the car has a damage which is not yet repaired.
* dateCreated - The date on which the eBay listing was created.
* nrOfPictures - The number of pictures in the ad.
* postalCode - The postal code for the location of the vehicle.
* lastSeenOnline - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

# Data reading and asessing

In [ ]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

In [ ]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


initial asessment of dataset:
* Five columns have missing values. 
* Five columns have `int64` data type, rest are `object` type. 
* At leats two columns (`price` and `odometer`) can be brought to `int64` data type. 
* The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

# Data processing and cleaning

We start our work with converting column names to snakecaseand. I could just assign `autos.columns` to list of desired column names, but it would be way to easy. I will create function which will convert CamelCase to snake_case.

In [ ]:
def Camel_to_snake(array):
    snake_case = []
    for name in array:
        snake_chars = []
        for char in name:
            if char.isupper():
                snake_chars.append("_"+char.lower())
            else:
                snake_chars.append(char)       
        snake_name = ''.join(snake_chars)
        snake_case.append(snake_name)
    return snake_case
autos.columns = Camel_to_snake(autos.columns)
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


As some of column headers are not descriptive enough or just awkward (mater of tastes) I'll replace those on spot using mapping.

In [ ]:
autos.rename(columns = {'abtest':'ab_test', 'year_of_registration':'registration_year', 'month_of_registration':'registration_month','not_repaired_damage':'unrepaired_damage','date_created':'ad_created','power_p_s':'power_ps'}, inplace = True)

Conclusion: anyways, I spent shitload of time trying to automize something very subjective. For my good future sake:
> remember - never pay to much attention to 'matter of taste' things while job  is not yet done. Focus on end goal and brush things you like to brush aftewards

Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for:

* Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
* Examples of numeric data stored as text which can be cleaned and converted.

In [ ]:
autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-30 17:37:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


`seller`, `offer_type`, `number_of_pictures` columns have only one value and are not of interest for analisis and can be dropped;

`price`, `odometer` columns have numeric data and can be transformed to be numeric.

In [ ]:
autos = autos.drop(['seller','offer_type','nr_of_pictures'], axis = 1)

In [ ]:
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')
autos['odometer'] = autos['odometer'].str.replace('km','')
autos['odometer'] = autos['odometer'].str.replace(',','')
autos.rename(columns = {'odometer':'odometer_km'}, inplace=True)

In [ ]:
autos['price']= autos['price'].astype(int)
autos['odometer_km']= autos['odometer_km'].astype(int)
autos.head()

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


Let's continue exploring the data, specifically looking for data that doesn't look right. We'll start by analyzing the `odometer_km` and `price` columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.

In [ ]:
print(autos['price'].unique().shape)     # to see how many unique values exist
print(autos['price'].describe())         # to see min, max and other info about collumn
print(autos['price'].value_counts().sort_index(ascending = True).head())     # to find outlined values which might be dropped
print(autos['price'].value_counts().sort_index(ascending = True).tail(15))   # to find outlined values which might be dropped

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
0    1421
1     156
2       3
3       1
5       2
Name: price, dtype: int64
265000      1
295000      1
299000      1
345000      1
350000      1
999990      1
999999      2
1234566     1
1300000     1
3890000     1
10000000    1
11111111    2
12345678    3
27322222    1
99999999    1
Name: price, dtype: int64


* I am really wondering what kind of car can cost more than `1.000.000$. I'll investigate dataset to find out and probably drop those cars from analysis.
* Regarding those cars with price around `1` - what I understood from comunity it is quite normal cause ebay is auction site. Those we will keep in place. 
* But those with `0` price I willl remove from analiys. 

In [ ]:
autos = autos.loc[autos['price'] != 0,:]
autos.loc[autos['price']>350000,:]

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
514,2016-03-17 09:53:08,Ford_Focus_Turnier_1.6_16V_Style,999999,test,kombi,2009,manuell,101,focus,125000,4,benzin,ford,nein,2016-03-17 00:00:00,12205,2016-04-06 07:17:35
2897,2016-03-12 21:50:57,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,11111111,test,limousine,1973,manuell,48,escort,50000,3,benzin,ford,nein,2016-03-12 00:00:00,94469,2016-03-12 22:45:27
7814,2016-04-04 11:53:31,Ferrari_F40,1300000,control,coupe,1992,NaN,0,NaN,50000,12,NaN,sonstige_autos,nein,2016-04-04 00:00:00,60598,2016-04-05 11:34:11
11137,2016-03-29 23:52:57,suche_maserati_3200_gt_Zustand_unwichtig_laufe...,10000000,control,coupe,1960,manuell,368,NaN,100000,1,benzin,sonstige_autos,nein,2016-03-29 00:00:00,73033,2016-04-06 21:18:11
22947,2016-03-22 12:54:19,Bmw_530d_zum_ausschlachten,1234566,control,kombi,1999,automatik,190,NaN,150000,2,diesel,bmw,NaN,2016-03-22 00:00:00,17454,2016-04-02 03:17:32
24384,2016-03-21 13:57:51,Schlachte_Golf_3_gt_tdi,11111111,test,NaN,1995,NaN,0,NaN,150000,0,NaN,volkswagen,NaN,2016-03-21 00:00:00,18519,2016-03-21 14:40:18
27371,2016-03-09 15:45:47,Fiat_Punto,12345678,control,NaN,2017,NaN,95,punto,150000,0,NaN,fiat,NaN,2016-03-09 00:00:00,96110,2016-03-09 15:45:47
37585,2016-03-29 11:38:54,Volkswagen_Jetta_GT,999990,test,limousine,1985,manuell,111,jetta,150000,12,benzin,volkswagen,ja,2016-03-29 00:00:00,50997,2016-03-29 11:38:54
39377,2016-03-08 23:53:51,Tausche_volvo_v40_gegen_van,12345678,control,NaN,2018,manuell,95,v40,150000,6,NaN,volvo,nein,2016-03-08 00:00:00,14542,2016-04-06 23:17:31
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,99999999,control,limousine,1999,automatik,224,s_klasse,150000,9,benzin,mercedes_benz,NaN,2016-03-22 00:00:00,73525,2016-04-06 05:15:30


Interesting to see Volkswagen Jetta in line with Ferrari F_40. I will drop all cars of this price segment. Those represent less than 0.03% of our data.

In [ ]:
autos = autos.loc[autos['price']<350001,:]

Now focus on `odometer_km` column

In [ ]:
autos['odometer_km'].value_counts(ascending=True) 

10000       253
20000       762
30000       780
40000       815
5000        836
50000      1012
60000      1155
70000      1217
80000      1415
90000      1734
100000     2115
125000     5057
150000    31414
Name: odometer_km, dtype: int64

`odometer_km` column looks completly normal. Values are rounded and probably those was filled from drop down list of some kind.

# Exploring date columns



There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler - string dtype
- `last_seen`: added by the crawler - string dtype
- `ad_created`: from the website - string dtype
- `registration_month`: from the website - int64 dtype
- `registration_year`: from the website int64 dtype

`date_crawled`, `last_seen` and `ad_created` columns needs to be converted to numeric type. Data representation in those columns are `YY-MM-DD HH:MM:SS`. I terms of necessity we will leave only `YY-MM-DD`. 

In [ ]:
autos['date_crawled'] = autos['date_crawled'].str[:10]
autos['last_seen'] = autos['last_seen'].str[:10]
autos['ad_created'] = autos['ad_created'].str[:10]
autos.head()

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26,79588,2016-04-06
1,2016-04-04,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04,71034,2016-04-06
2,2016-03-26,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26,35394,2016-04-06
3,2016-03-12,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12,33729,2016-03-15
4,2016-04-01,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01,39218,2016-04-01


In [ ]:
autos['date_crawled'].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

I have no idea how it gonna help in analysis. But ok. Values in `date_crawled` evenly spreaded trough the march and beginning of april with minor deviation from average of `0,03`. 

In [ ]:
autos['last_seen'].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

`last_seen` column is evenly spreaded over the same time span. Most likely last seen section is the date when the car was sold.

In [ ]:
autos['ad_created'].value_counts(normalize=True, dropna=False).sort_index().tail(40)

2016-02-28    0.000206
2016-02-29    0.000165
2016-03-01    0.000103
2016-03-02    0.000103
2016-03-03    0.000865
2016-03-04    0.001483
2016-03-05    0.022897
2016-03-06    0.015320
2016-03-07    0.034737
2016-03-08    0.033316
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21    0.037579
2016-03-22    0.032801
2016-03-23    0.032060
2016-03-24    0.029280
2016-03-25    0.031751
2016-03-26    0.032266
2016-03-27    0.030989
2016-03-28    0.034984
2016-03-29    0.034037
2016-03-30    0.033501
2016-03-31    0.031875
2016-04-01    0.033687
2016-04-02    0.035149
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, dtype: float64

`ad_created` column have a discernable pattern. The older the ad, the less frequently its occurance, which can be explained. The longer ad persist, the more people looked on it and the higher probability to be sold. Probably the remaning offers are so bad, that nobody have will to make offer.

Last - lets explore `registration_year` column.

In [ ]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

From above we can conclude, that there are no missing values, but there are some odd values like `1000` or `9999` which are min and max values.

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [ ]:
autos['registration_year'].between(1900,2016).value_counts()

True     46681
False     1884
Name: registration_year, dtype: int64

Values which doesn't fall in specified gap represent 3,9% of all values and can be ommited.

# `Brand` column exploration


It is natural will to investigate dataset in regard of `brand` type. Let's see how many unique values we have, it might be that we need to uniform brand names to make analisys possible.

In [ ]:
autos['brand'].unique()

array(['peugeot', 'bmw', 'volkswagen', 'smart', 'ford', 'chrysler',
       'seat', 'renault', 'mercedes_benz', 'audi', 'sonstige_autos',
       'opel', 'mazda', 'porsche', 'mini', 'toyota', 'dacia', 'nissan',
       'jeep', 'saab', 'volvo', 'mitsubishi', 'jaguar', 'fiat', 'skoda',
       'subaru', 'kia', 'citroen', 'chevrolet', 'hyundai', 'honda',
       'daewoo', 'suzuki', 'trabant', 'land_rover', 'alfa_romeo', 'lada',
       'rover', 'daihatsu', 'lancia'], dtype=object)

OMG!!! There is `lada` in the list. Let's find out the percentage of German, Asian, European, US and Russian car brands are represented. Belonging of brand to certain country or geografical location is disputable. Some "seem to be" EU car brands are actualy have Indian or chinese owners. 

In [ ]:
origin = {'Germany':['trabant','porsche','opel','bmw','volkswagen','smart','mercedes_benz','audi'],
        'Europe':['rover','land_rover','alfa_romeo','skoda','citroen','peugeot','seat','renault','mini','saab','volvo','jaguar','fiat'],
        'US':['ford','chrysler','jeep','chevrolet'],
        'Asia':['daihatsu','lancia','suzuki','mazda','subaru','kia','toyota','hyundai','honda','daewoo','dacia','nissan','mitsubishi'],
        'Russia':'lada'}
percentages = {'german_cars' : (sum([1 for each in autos['brand'] if each in origin['Germany']])/len(autos['brand'])),
'russian_cars' : (sum([1 for each in autos['brand'] if each in origin['Russia']])/len(autos['brand'])),
'asian_cars' : (sum([1 for each in autos['brand'] if each in origin['Asia']])/len(autos['brand'])),
'US_cars' : (sum([1 for each in autos['brand'] if each in origin['US']])/len(autos['brand'])),
'european_cars' : (sum([1 for each in autos['brand'] if each in origin['Europe']])/len(autos['brand']))}
percentages

{'german_cars': 0.6332955832389581,
 'russian_cars': 0.0005971378564810048,
 'asian_cars': 0.09218573046432615,
 'US_cars': 0.08100483887573355,
 'european_cars': 0.18321836713682693}

We can see that russian autos are not so popular on eBay. Let think that `lada` is so good that nobody want to sell. More than 81% of cars on German eBay represent european manufacturers. Germany itself is far ahead in terms of representation on local market. 

Lets now find top 10 of the most popular brands.

In [ ]:
autos['brand'].value_counts(ascending = False).head(10)

volkswagen       10336
opel              5277
bmw               5274
mercedes_benz     4652
audi              4168
ford              3382
renault           2325
peugeot           1430
fiat              1262
seat               920
Name: brand, dtype: int64

No surprises. Germans o n top three, and `volkswagen` almost twice more representative. As Russian leaving in Kaliningrad I can confirm that mosr of cars on our roads are old germans. There is prejudice that average man drives old Passat in Kaliningrad. Lets try to fin how many Passats are out there on eBay.

In [ ]:
autos.loc[autos['model'] == 'passat', 'model'].value_counts()


passat    1383
Name: model, dtype: int64

Only 1383 ((( Could be more. Let see the most popular models out there.

In [ ]:
autos['model'].value_counts(ascending = False).head(10)

golf        3900
andere      3443
3er         2686
polo        1688
corsa       1681
astra       1410
passat      1383
a4          1256
c_klasse    1161
5er         1150
Name: model, dtype: int64

Lets investigate what kind of `andere` model is?

In [ ]:
autos.loc[autos['model'] == 'andere', :]

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26,79588,2016-04-06
19,2016-03-17,mazda_tribute_2.0_mit_gas_und_tuev_neu_2018,4150,control,suv,2004,manuell,124,andere,150000,2,lpg,mazda,nein,2016-03-17,40878,2016-03-17
45,2016-03-12,Toyota_Starlet_1.3,1000,test,kleinwagen,1995,manuell,75,andere,150000,3,benzin,toyota,nein,2016-03-12,74597,2016-04-07
52,2016-03-25,Senator_A_3.0E_Karosserie_restauriert_m._viele...,3500,test,limousine,1985,NaN,0,andere,5000,0,benzin,opel,nein,2016-03-25,63500,2016-04-07
70,2016-03-16,Saab_9_3_2.0i_Cabrio_SE,1750,control,cabrio,1999,manuell,131,andere,150000,4,benzin,saab,nein,2016-03-16,81829,2016-03-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49920,2016-03-28,Mazda_323_F,700,test,kleinwagen,2000,manuell,65,andere,150000,3,NaN,mazda,NaN,2016-03-28,36205,2016-04-06
49944,2016-03-15,SAAB_9_3__2_0t_SE__Automatik__Tempomat__Regens...,1000,control,limousine,2000,automatik,150,andere,150000,6,benzin,saab,nein,2016-03-15,4105,2016-04-02
49962,2016-03-14,Mitsubishi_Space_Star_1__3_L__Bj_2004_Standhei...,2200,test,limousine,2004,manuell,82,andere,150000,4,benzin,mitsubishi,ja,2016-03-14,45481,2016-03-15
49968,2016-04-01,Mercedes_Benz_190_D_2.5_Automatik,2100,test,limousine,1986,automatik,90,andere,150000,9,diesel,mercedes_benz,nein,2016-04-01,40227,2016-04-05


Google translate says that `andere` stands for *other*

Let's be more professional and follow advice of project guide. On next step I will agregate data for German car brands and will calculate mean price for those (`bmp` - brand mean price).

In [ ]:
bmp = {name:round(sum(autos.loc[autos['brand']== name,'price'])/autos[autos['brand']==name].shape[0]) for name in origin['Germany']} 
bmp

{'trabant': 1771,
 'porsche': 45624,
 'opel': 2945,
 'bmw': 8261,
 'volkswagen': 5332,
 'smart': 3518,
 'mercedes_benz': 8536,
 'audi': 9213}

From data agregated above we can conclude that `volkswagen` and `smart` form middle price class, while `opel` and `trabant` form low price class. Now lets find the average price for Volkswagen passat.

In [ ]:
volkswagen_passat = sum(autos.loc[autos['model']=='passat', 'price'])/autos[autos['model']=='passat'].shape[0]
round(volkswagen_passat)

4919

Now we can see why `passat` is the best car ever. 

# Milage exploration

Project guide suggests the hypotesis that there is dependance between `price` and `milage` which is quite obvious. This is obvious within homogenous data, meaning that this will perfectly work for Volkswagen Passat data set. It doesn't seem to me as valide hypotesis for dataframe with all kinds of brands.
Nevertheless we will try and find out later.

`bmm` - stands for brand mean milage

`year_avg` - average year of registration

In [ ]:
bmm = {name:round(sum(autos.loc[autos['brand']== name,'odometer_km'])/autos[autos['brand']==name].shape[0]) for name in origin['Germany']} 
year_avg = {name:round(sum(autos.loc[autos['brand']== name,'registration_year'])/autos[autos['brand']==name].shape[0]) for name in origin['Germany']}

To be able to assess data we will combine dictionaries into dataframe.

In [ ]:
bmm_series = pd.Series(bmm)
bmp_series = pd.Series(bmp)
year_avg_series = pd.Series(year_avg)
hypothesis = pd.DataFrame(bmm_series, columns = ['mean_milage'])
hypothesis['mean_price'] = bmp_series
hypothesis['year_avg'] = year_avg_series
hypothesis

,mean_milage,mean_price,year_avg
trabant,54412,1771,1970
porsche,96760,45624,2002
opel,129383,2945,2006
bmw,132683,8261,2003
volkswagen,128897,5332,2005
smart,100512,3518,2006
mercedes_benz,130796,8536,2005
audi,129493,9213,2005


Strange picture arose. `BMW` and `Mercedes-benz` have highest average milage, but price of those are higher than for example `smart` which is in general has less miles behind. There is no clear correlation between milage and price. Because german cars - they are like a vine. Getting better with age.